# SSD512 Inference Tutorial

This is a brief tutorial that shows how to use a trained SSD512 for inference on the Pascal VOC datasets. It is the same as the SSD300 inference tutorial but with all parameters preset for SSD512 for Pascal VOC. If you'd like more detailed explanations on how to use the model generally, please refer to [`ssd300_training.ipynb`](https://github.com/pierluigiferrari/ssd_keras/blob/master/ssd300_training.ipynb)

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from imageio import imread
import numpy as np
from matplotlib import pyplot as plt

from models.keras_ssd512 import ssd_512
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

from os import listdir
from os.path import isfile, join
from exif import Image

import time

%matplotlib inline

In [ ]:
# Set the image size.
img_height = 512
img_width = 512

## 1. Load a trained SSD

Either load a trained model or build a model and load trained weights into it. Since the HDF5 files I'm providing contain only the weights for the various SSD versions, not the complete models, you'll have to go with the latter option when using this implementation for the first time. You can then of course save the model and next time load the full model directly, without having to build it.

You can find the download links to all the trained model weights in the README.

### 1.1. Build the model and load trained weights into it

In [ ]:
# 1: Build the Keras model

K.clear_session() # Clear previous models from memory.

model = ssd_512(image_size=(img_height, img_width, 3),
                n_classes=20,
                mode='inference',
                l2_regularization=0.0005,
                scales=[0.07, 0.15, 0.3, 0.45, 0.6, 0.75, 0.9, 1.05], # The scales for MS COCO are [0.04, 0.1, 0.26, 0.42, 0.58, 0.74, 0.9, 1.06]
                aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5]],
               two_boxes_for_ar1=True,
               steps=[8, 16, 32, 64, 128, 256, 512],
               offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
               clip_boxes=False,
               variances=[0.1, 0.1, 0.2, 0.2],
               normalize_coords=True,
               subtract_mean=[123, 117, 104],
               swap_channels=[2, 1, 0],
               confidence_thresh=0.5,
               iou_threshold=0.45,
               top_k=200,
               nms_max_output_size=400)

# 2: Load the trained weights into the model.

# TODO: Set the path of the trained weights.
weights_path = 'path/to/trained/weights/VGG_VOC0712_SSD_512x512_iter_120000.h5'

model.load_weights(weights_path, by_name=True)

# 3: Compile the model so that Keras won't complain the next time you load it.

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

Or

### 1.2. Load a trained model

In [ ]:
# TODO: Set the path to the `.h5` file of the model to be loaded.
model_path = R'path/to/trained/model.h5'

# We need to create an SSDLoss object in order to pass that to the model loader.
ssd_loss = SSDLoss(neg_pos_ratio=3, n_neg_min=0, alpha=1.0)

K.clear_session() # Clear previous models from memory.

model = load_model(model_path, custom_objects={'AnchorBoxes': AnchorBoxes,
                                               'L2Normalization': L2Normalization,
                                               'DecodeDetections': DecodeDetections,
                                               'compute_loss': ssd_loss.compute_loss})

## 2. Load some images

Images should not have any orientation in the file metadata

In [ ]:
image_path = R'path/'

fichiers = [f for f in listdir(image_path) if isfile(join(image_path, f))]

for file in fichiers:
    with open(image_path + '\\' + file, 'rb') as image_file:
        my_image = Image(image_file)

    if 'orientation' in my_image.list_all():
        del my_image.orientation

    with open(image_path + '\\' + file, 'wb') as new_image_file:
        new_image_file.write(my_image.get_file())

Load some images for which you'd like the model to make predictions.

In [ ]:
image_path = R'path/'

fichiers = [f for f in listdir(image_path) if isfile(join(image_path, f))]

orig_images = [] # Store the images here.
input_images = [] # Store resized versions of the images here.

for file in fichiers:
    img_path = image_path + '\\' + file
    orig_images.append(imread(img_path))
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img = image.img_to_array(img) 
    input_images.append(img)
    
input_images = np.array(input_images)

In [ ]:
input_images.shape

## 3. Make and visualize predictions

`y_pred` contains a fixed number of predictions per batch item (200 if you use the original model configuration), many of which are low-confidence predictions or dummy entries. We therefore need to apply a confidence threshold to filter out the bad predictions. Set this confidence threshold value how you see fit.

In [ ]:
pred_time = time.time()

# Makes predictions on images
y_pred = model.predict(input_images)
print("--- Pred time : %s seconds ---" % (time.time() - pred_time))

colors = plt.cm.hsv(np.linspace(0, 1, 3)).tolist()
colors = ['black', 'red', 'blue']
classes = ['background',
           'aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat',
           'chair', 'cow', 'diningtable', 'dog',
           'horse', 'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor']

start_time = time.time()

time_list = []

for i in range(len(orig_images)):
    image_time = time.time()
    # Decode the predictions
    y_pred_decoded = decode_detections(y_pred, confidence_thresh=0.5, iou_threshold=0.5, top_k=1000, normalize_coords=True, img_height=orig_images[i].shape[0], img_width=orig_images[i].shape[1])
    time_list.append(time.time() - image_time)
    print("--- Image", i, " time %s seconds ---" % (time.time() - image_time))
    plt.figure(figsize=(20,12))
    plt.imshow(orig_images[i])

    current_axis = plt.gca()
    for box in y_pred_decoded[i]:
        xmin = box[2]
        ymin = box[3]
        xmax = box[4]
        ymax = box[5]
        color = colors[round(box[0])]
        label = '{}: {:.2f}'.format(classes[round(box[0])], box[1])
        current_axis.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color=color, fill=False, linewidth=2))
        current_axis.text(xmin, ymin, label, size='x-large', color='white', bbox={'facecolor':color, 'alpha':1.0})
    plt.show()

print("--- Global time : %s seconds ---" % (time.time() - start_time))